<a href="https://www.kaggle.com/code/gizemnalbantarslan/time-series-w-sar-max?scriptVersionId=199070058" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import itertools
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.tsa.statespace.sarimax import SARIMAX
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/time-series-starter-dataset/Month_Value_1.csv")
df.head()

In [ ]:
df['Period']=pd.to_datetime(df['Period'],format="%d.%m.%Y")
df.head()

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())

In [ ]:
check_df(df)

Our data set contains 32 rows of empty data. We reserve this field as test data that we will predict. We will separate the remaining data as validation and train set and build our model on these data. We make sure that our validation data has the same size as the test data.

In [ ]:
def create_date_features(df, date_column):
    df['month'] = df[date_column].dt.month
    df['day_of_month'] = df[date_column].dt.day
    df['day_of_year'] = df[date_column].dt.dayofyear
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['year'] = df[date_column].dt.year
    df["is_wknd"] = df[date_column].dt.weekday // 4
    df['is_month_start'] =df[date_column].dt.is_month_start.astype(int)
    df['is_month_end'] = df[date_column].dt.is_month_end.astype(int)
    df['quarter'] = df[date_column].dt.quarter
    df['is_quarter_start'] = df[date_column].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df[date_column].dt.is_quarter_end.astype(int)
    df['is_year_start'] = df[date_column].dt.is_year_start.astype(int)
    df['is_year_end'] = df[date_column].dt.is_year_end.astype(int)
    return df

In [ ]:
train=df.loc[0:31,]
train.head()
train.tail()

In [ ]:
print(train.isnull().sum())

In [ ]:
create_date_features(train, "Period")

In [ ]:
val = df.loc[32:63,]
val.head()
val.tail()

In [ ]:
print(val.isnull().sum())

In [ ]:
create_date_features(val, "Period")

First, we test the hypothesis for stationarity.

In [ ]:
def is_stationary(y):

    # "HO: Non-stationary"
    # "H1: Stationary"

    p_value = sm.tsa.stattools.adfuller(y)[1]
    if p_value < 0.05:
        print(F"Result: Stationary (H0: non-stationary, p-value: {round(p_value, 3)})")
    else:
        print(F"Result: Non-Stationary (H0: non-stationary, p-value: {round(p_value, 3)})")

is_stationary(train["Sales_quantity"])

From our stationarity hypothesis, we now know that we cannot use "SES", "AR", "MA" or "ARMA". According to the seasonality component, we can use ARIMA or SARIMA models.

In [ ]:
plt.plot(train.year, train.Sales_quantity)
plt.show()

As we can see from the chart, seasonality is also in the trend. So we will continue with the SARIMA model.

**SARIMA(p, d, q): (Seasonal Autoregressive Integrated Moving-Average)**

In [ ]:
model = SARIMAX(train["Sales_quantity"], order=(1, 0, 1), seasonal_order=(0, 0, 0, 12))
sarima_model = model.fit(disp=0)

In [ ]:
#  From the train set, we get the predicted values for the validation set.
y_pred_val = sarima_model.get_forecast(steps=32)
y_pred = y_pred_val.predicted_mean
y_pred = pd.Series(y_pred, index=val.index)

In [ ]:
def plot_sales_quant(train, test, y_pred, title):
    mae = mean_absolute_error(test, y_pred)
    train.plot(legend=True, label="TRAIN", title=f"{title}, MAE: {round(mae,2)}")
    test.plot(legend=True, label="TEST", figsize=(6, 4))
    y_pred.plot(legend=True, label="PREDICTION")
    plt.show()

In [ ]:
plot_sales_quant(train["Sales_quantity"], val["Sales_quantity"], y_pred, "SARIMA")

We can see from the graph that we made a bad forecast. Therefore, we need to optimize the hyperparameters.

In [ ]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
def sarima_optimizer_aic(train, pdq, seasonal_pdq):
    best_aic, best_order, best_seasonal_order = float("inf"), None, None
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                sarimax_model = SARIMAX(train, order=param, seasonal_order=param_seasonal)
                results = sarimax_model.fit(disp=0)
                aic = results.aic
                if aic < best_aic:
                    best_aic, best_order, best_seasonal_order = aic, param, param_seasonal
                print('SARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, aic))
            except:
                continue
    print('SARIMA{}x{}12 - AIC:{}'.format(best_order, best_seasonal_order, best_aic))
    return best_order, best_seasonal_order

In [ ]:
best_order, best_seasonal_order = sarima_optimizer_aic(train["Sales_quantity"], pdq, seasonal_pdq)

In [ ]:
best_order, best_seasonal_order

Thus we find combinations of best_order, best_seasonal_order. Now we will build our final model with these combinations.

**FINAL MODEL**

Previously, we divided our dataset into train, validation and test. We made optimizations over train and validation sets and set up the best parameter combinations. Now we will split our initial "df" dataset into train and test and predict on the test set with the final model. 

In [ ]:
df = pd.read_csv("/kaggle/input/time-series-starter-dataset/Month_Value_1.csv")
df['Period']=pd.to_datetime(df['Period'],format="%d.%m.%Y")
df.head()

In [ ]:
train=df.loc[0:63,]
train.head()
train.tail()

In [ ]:
test=df.loc[64:,]
test.head()

In [ ]:
model = SARIMAX(train["Sales_quantity"], order=best_order, seasonal_order=best_seasonal_order)
sarima_final_model = model.fit(disp=0)

In [ ]:
feature_predict = sarima_final_model.get_forecast(steps=32)
feature_predict = feature_predict.predicted_mean

Thus, we get the empty "Sales_quantity" data for the test data.